# <center>Big Data &ndash; Exercises</center>
## <center>Fall 2025 &ndash; Week 3 &ndash; ETH Zurich</center>

## Introduction
This week we will cover mostly theoretical aspects of Hadoop and HDFS and we will discuss advantages and limitations of different storage models.

#### What is Hadoop?
Hadoop provides a **distributed file system** and a
**framework for the analysis and transformation** of very **large**
data sets using the MapReduce paradigm.

Several components are part of this framework. In this course we will study HDFS, MapReduce and HBase while this exercise focuses on HDFS and storage models.


| *Component*                |*Description*  |*First developer*  |
|----------------------------------------------|---|---|
| **HDFS**                  |Distributed file system  |Yahoo!  |
| **MapReduce**   |Distributed computation framework   |Yahoo!  |
| **HBase**           | Column-oriented table service  |Powerset (Microsoft)  |
| Pig  | Dataflow language and parallel execution framework  |Yahoo!   |
| Hive            |Data warehouse infrastructure   |Facebook  |
| ZooKeeper    |Distributed coordination service   |Yahoo!  |
| Chukwa  |System for collecting management data   |Yahoo!  |
| Avro                |Data serialization system   |Yahoo! + Cloudera  |

## 1. The Hadoop Distributed File System
### 1.1 &ndash; State which of the following statements are true:

1. The HDFS namespace is a hierarchy of files and directories. TRUE

1. All blocks of a file in HDFS are stored sequentially on a single disk for faster access. FALSE

1. A client wanting to write a file into HDFS, first contacts the NameNode, then sends the data to it. The NameNode will write the data into multiple DataNodes in a pipelined fashion. FALSE

1. A DataNode may execute multiple application tasks for different clients concurrently. TRUE

1. If the block size is set to 64 megabytes, storing a file of 80 megabytes will actually require 128 megabytes of physical memory (2 blocks of 64 megabytes each). FALSE -last block is not the fixed size

1. The cluster can have thousands of DataNodes and tens of thousands of HDFS clients per cluster. TRUE

1. HDFS NameNodes keep the namespace in DRAM and on disk. TRUE

1. The locations of block replicas are part of the persistent checkpoint that the NameNode stores in its local file system. FALSE

### 1.2 &ndash; A typical filesystem block size is 4096 bytes. How large is a block in HDFS? List at least two advantages of such choice.

A Block in HDFS is normally 128 MB but this can be changed by the user. This size is perfect for transferring object through a network and makes it easier to move them around without latency restrictions. The last block can vary in size so no memory is wasted.

From solutions:

The typical size for a block is either 64 or 128 megabytes. A large block size offers several important advantages. 

1. It reduces the size of the metadata stored on the NameNode. This allows us to keep the metadata in memory.

2. It minimizes the proportion of time spent seeking (i.e. moving the read/write head to the correct position on the disk). The seek time is a constant overhead for each block of data you want to access. Using large blocks ensures that after the initial seek, the system can continuously transfer large amounts of data at the disk's full transfer rate.

3. It reduces clients' need to interact with the NameNode because reads and writes on the same block require only one initial request to the NameNode for location information. The reduction is significant for workloads where applications mostly read and write large files sequentially. 

4. Since on a large block, a client is more likely to perform many operations on the given block, it can reduce network overhead by keeping a persistent TCP connection to the DataNode over an extended period of time. 

### 1.3 &ndash; Answer the following questions:

1. Why does the hardware cost of HDFS grow linearly with the amount of data? Because Data is spreaded homogenuosly across the cluster so anytime there is need of more hardware it'll be linear w.r.t data that needs to be stored

1. Why is the NameNode a single point of failure? Because the architecture is centralized and a client before accessing a datanode has to pass through the Namenode, many actions are also done from the Namenode, if this fails, the archotecture fails. ALL DATA LOST.

1. How is this problem addressed? In different ways, there is the addition of standbyNodes which give the system a chance to autonomally restart if a failure occurs, Snapshots save the state of the Namenode every few hours while editlog keep track of the changes of every update and they merge resetting edit log called Checkpoint this grows very fast and delays the restarting so StandbyNodes where introduces such as Secondary Namenode that is effectively a second Namenode in memory ready to enter the game as soon as something goes wrong. Ord Federated Namenodes which are different nademodes taking care of different namespaces that do not intersect

### 1.4 &ndash; Scalability, Durability and Performance
Explain how HDFS accomplishes the following requirements:

1. Scalability. New nodes can enter whenever memory is needed, scales out. Files are divided into blocks and distributes, this may scale to many files. Can scale beyond tens of Petabytes and grows linearly

1. Durability. Through STandbyNodes or Federated Namenodes, fault tolerance is there. And also creating copies of blocks minimum 3 but number can be changed by the user.

1. High sequential read/write performance. DataNodes can work sequentially and data is sent in smaller packets such as 64kbytes so that pipelines are fast and also streams are useful to hide from user as much as possible, the user simply sees data flowing around. Files split in block make access cocurrent and fast

## 2. File I/O operations and replica management.

### 2.1 &ndash; File read/write data flow.
To get an idea of how data flows between the client interacting with HDFS, consider a diagram below which shows the main components of HDFS. 

<img src="https://polybox.ethz.ch/index.php/s/R7hg8x7YEyTFPvD/download" width="600">

1. Draw the main sequence of events when a client writes a file to HDFS.
    - Client request Namenode where to write
    - Nademode sends back a list of Datanodes where to write sorted by distance from the client.
    - The blocks have a lock until the writing is not over
    - Client connect with the first Datanode in the list and instructs the Datanodes to create a pipeline with other Dtanodes in the list
    - CLient start sending data in smaller packets of 64Kbytes so that if some are lost can be resent
    - Datanodes acknowledge always until the data is over
    - Once its done Client connect with Namenode saying its done 
    - Namenode waits for Datanodes heartbeart to check everything is ok
    - Send ultimate ackoledge releasing lock

    Requesting block to write up until last block has been written is repeated per block!!! once file is done the asks namenode for release of lock

2. Draw the main sequence of events when a client reads a file from HDFS.
    - Requests Namenode
    - Namedone send back list of Datanodes with position of blocks sorted by distance to client
    - CLient connect to Datanodes in the list from the closest and goes down the list if not available
    - Request data from Datanodes and to download the blocks, data flows via streams.
    - it can also download multiHDFS blocks and store the file locally if fits
    
3. Why do you think a client writes data directly to datanodes instead of sending it through the namenode?
    Because Datanodes can works concurrently and there is where the content is stored, the Namenode is used only to manage the system 

### 2.2 &ndash; Synchronous and asynchronous operations

For each of the operation below, specify whether it is conducted synchronously or asynchronously.

1. Writing a 128MB data block from the client to the cluster. Synchronously

2. Sending and receiving BlockReports. Asynchronously

3. Rebalancing blocks across DataNodes. Asynchronously

4. NameNode's checking for the minimal replication of a file that the client just wrote. Synchrounously

5. Replicating a file written by the client to achieve the target replication by the NameNode. Asynchronously

### 2.3 &ndash; Replication policy
Assume your HDFS cluster is made of 3 racks, each containing 3 DataNodes. Assume also that HDFS is configured to use a block size of 100 megabytes and that a client is connecting from outside the datacenter (therefore no DataNode is privileged). 

1. The client uploads a file of 150 megabytes. Draw in the picture below a possible block configuration according to the default HDFS replica policy. How many replicas are there for each block? Where are these replicas stored?

Replication policies states at most 1 replica per node and at most 2 per rack. In this case File is 150 megabytes so we need 2 blocks, first 100megabyte and 50megabyte second. We can place 

    - Rack 1 : Node 1: 1 Replica of 100MB, Node 2 Replica of 50MB
    - Rack 2 : Node 4: 1 replica of 100MB, Replica of 50MB; Node 5 Replica of 50MB, Replica 100MB
    - Rack 3 : Node 7: 1 Replica of 100MB, Node 8 Replica of 50MB

This makes it 3 so its not underreplicated and constraints are satisfied

1. Can you find a different policy that, using the same number of replicas, improves the expected availability of a block? Does your solution show any drawbacks?
We can try to put as many replicas as possible on the same rack as the node where clinet process is ran.

1. Referring to the picture below, assume a block is stored on Node 3, as well as on Node 4 and Node 5. If this block of data has to be processed by a task running on Node 6, which of the three replicas will be actually read by Node 6? 
    Either 5 or 4 because they are closer just distance of 2.

<img src="https://polybox.ethz.ch/index.php/s/lRzwDdtmytzyDRR/download" width="500">

### 2.4 &ndash; Network topology.

HDFS estimates the network bandwidth between two nodes by their distance. The distance from a node to its parent node is assumed to be one. A distance between two nodes can be calculated by summing up their distances to their closest common ancestor. A shorter distance between two nodes means that a greater bandwidth can be utilized to transfer data. Consider a diagram of a possible Hadoop cluster over two datacenters below.

<img src="https://polybox.ethz.ch/index.php/s/Mk2kI7dkKZNrxul/download" width="700">

Calculate following distances using the distance rule explained above:

1. Node 0 and Node 1 -> distance 2
2. Node 0 and Node 2 -> distance 4
3. Node 1 and Node 4 -> distance 6
4. Node 4 and Node 5 -> distance 4
5. Node 2 and Node 3 -> distance 2
6. Two processes of Node 1 -> distance 0

## 3. Storage models
### 3.1 &ndash; List two differences between Object Storage and Distributed File Systems.

The storing of the data. Here a hierchy exists while in object stores it does not it is a flat list of key-value objects .
    
DFS can store large quantity of huge files while object stores can store huge quantity of large files

### 3.2 &ndash; Compare Object Storage and HDFS. For each of the following use cases, say which technology better fits the requirements.

1. Store Netflix movie files in such a way that they are accessible from many client applications at the same time. Object Storage

1. Store experimental and simulation data from CERN. HDFS

1. Store the auto-backups of iPhone/Android devices. Oject Stores

1. Store vast amounts of telescope data for astronomical pattern detection. HDFS

## 4. Working with Docker-Hadoop (Optional) - Would love to do it but exam is close

Build and run the Hadoop docker image by typing `docker-compose up -d` in the `exercise03` directory. If completed successfully, you should be able to browse [`http://localhost:9870/`](http://localhost:9870/) and visualize the web interface of the daemon which should look similar to the following image. In the `Datanodes` tab you should see a single operating datanode.

<img src="https://polybox.ethz.ch/index.php/s/LpWcGWZeU5mipBK/download" width="800">


### Connecting to containers  

Each Hadoop cluster is set up in one of the three supported modes:

- Local (Standalone) Mode
- Pseudo-Distributed Mode
- Fully-Distributed Mode

By default Hadoop runs in Local Mode but we will run it in the *Pseudo-Distributed Mode*. This will allow you to run Hadoop on a single-node (your computer) simulating a distributed file system, with datanode and namenode running in separate containers. For this excercise you will only need to connect to `namenode` and `datanode` containers. To connect to namenode container can use the Docker dashboard interface by navigating to `docker-hadoop` app, and selecting `CLI` option from the `namenode` container (see image below).

<img src="https://polybox.ethz.ch/index.php/s/Hdlyhagx3JWbLBy/download" width="700">

Alternatively, you can run `docker exec -it namenode /bin/bash` in a terminal. To connect to a datanode, you can similarly find it in the dashboard or run `docker exec -it namenode /bin/bash` in the terminal. Both approaches will give you shell access on the corresponding container. 

### 4.1 &ndash; Upload a file into HDFS

Connect to the namenode by `docker exec -it namenode /bin/bash`.

Pick an image file on your computer (or you can also download a random one) and try to upload it to HDFS. You may need to create an empty directory before uploading. (Check [here](https://hadoop.apache.org/docs/current/hadoop-project-dist/hadoop-common/FileSystemShell.html) for help.)

1. Which command do you use to upload from the local file system to HDFS?

1. Which information can you find if you use `Utilities -> Browse the file system` in the daemon web interface?

### 4.2 &ndash; Local File System

1. ```bash
   docker cp docker-compose.yml namenode:docker-compose.yml 
   ```
Then, use HDFS commands to create a directory, copy the `docker-compose.yml` file from your local file system to HDFS. Use `cat` to check if the file is the same on the local and distributed systems. 

   *Hint:* you may use the following HDFS commands `-mkdir` for directory, `-copyFromLocal` for uploading the file, and `-cat` for printing them. You may have to first use `docker cp` to copy to file into the namenode container.

2. Try to locate the file on a datanode. To connect to a datanode by running:

   ```bash
   docker exec -it datanode /bin/bash
   ```

   This will give you shell access to the data node machine. cd into `/hadoop/dfs/data/current/` directory and follow the directories until there are only files. Can you check if the file contents are the same as the one you uploaded? Use `ls -l` to check the size of the file size on the local 

3. Now try to upload a file to HDFS that is ~150MB. On Unix-based system you can also generate such a file filled with zero using:

   ```bash
   dd if=/dev/zero of=zeros.dat bs=1M count=150
   ```

   How many blocks the file is split into?

### 4.3 Demystifying FsImage, Edits and Checkpoint

When the NameNode starts up, or a checkpoint is triggered by a configurable threshold:

- It reads the FsImage and EditLog from disk.
- It applies all the transactions from the EditLog to the in-memory representation of the FsImage.
- It flushes out this new version into a new FsImage on disk.
- It truncates the old EditLog because its transactions have been applied to the persistent FsImage.

A checkpoint can be triggered:

> at a given time interval (dfs.namenode.checkpoint.period) expressed in seconds,
> or after a given number of filesystem transactions have accumulated (dfs.namenode.checkpoint.txns).

If both of these properties are set, the first threshold to be reached triggers a checkpoint.

1. Query the configuration file

   - `hdfs getconf -confKey dfs.namenode.checkpoint.period`
   - `hdfs getconf -confKey dfs.namenode.checkpoint.txns`
   - The fsimage & edit logs location `hdfs getconf -confKey dfs.namenode.name.dir`, I get something like `file:///hadoop/dfs/name`
   - Find the fsimage and edit logs in the `current` directory. They must be named like `fsimage_0000000000000000000` & `edits_inprogress_0000000000000000001` 
   - Output edits `hdfs oev -p xml -i /hadoop/dfs/name/current/edits_inprogress_0000000000000000001 -o edits.xml `
   - Output fsimage `hdfs oiv -p XML -i /hadoop/dfs/name/current/fsimage_0000000000000000000 -o fsimage.xml`

2. Can you make sense of the outputs?

### 4.4 Changing the Block Size

As explained in the tutorials, to change HDFS configurations you edit `etc/hadoop/core-site.xml` and `etc/hadoop/hdfs-site.xml`. In the docker app, you can modify the variables in the `hadoop.env`. For example, in the following line,

```bash 
# hadoop.env 
CORE_CONF_fs_defaultFS=hdfs://namenode:9000
```

`CORE_CONF` corresponds to `core-site.xml`. The second part `fs_defaultFS=hdfs://namenode:9000` will be transformed into:

```xml
<property>
    <name>fs.defaultFS</name><
    value>hdfs://namenode:9000
    </value>
</property>
```

For more details [see here](https://github.com/big-data-europe/docker-hadoop).

Try changing the default block size of HDFS to see its affect on read & write performance. You can change the block size by modifying the follwoing line in `hadoop.env`: `HDFS_CONF_dfs_block_size=1048576` The value `1048576` determines the block size in bytes, which in this case is `2^20 bytes` or 1 megabytes.

> **_NOTE:_** that for these configuration changes to take effect you must restart the docker app!

1. Create a file with size ~150MB and uploade the file to HDFS. Check number of blocks via the Web interface. 

2. For each of the following block sizes 1048576, 134217728, measure the time to transfer from local to HDFS, from HDFS to local, and removing the file. 

You can run the following commands 
```bash
time hadoop fs -copyFromLocal zeros.dat /myfolder/zeros.dat
time hadoop fs -get /myfolder/zeros.dat zeros.dat
time hadoop fs -rm /myfolder/zeros.dat
```
Can you make sense of the results?

> **_NOTE:_** make sure to remove the files before uploading them, so that no caching will distort the measurements